In [76]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [4]:
!pip install kss
#한국어 문장 분리모델

     |████████████████████████████████| 71kB 3.9MB/s 


In [53]:
import os
import kss
import re
import pandas as pd
import numpy as np
import konlpy
from tqdm import tqdm

In [6]:
# 형태소 기반 토크나이징 (Konlpy)
!python3 -m pip install konlpy
# 다른 os 설치 방법 및 자세한 내용은 다음 참고: https://konlpy.org/ko/latest/install/#id1
!bash <(curl -s https://raw.githubusercontent.com/konlpy/konlpy/master/scripts/mecab.sh)

     |████████████████████████████████| 19.4MB 1.4MB/s 
     |████████████████████████████████| 460kB 50.6MB/s 
     |████████████████████████████████| 92kB 7.1MB/s 
  Found existing installation: beautifulsoup4 4.6.3
    Uninstalling beautifulsoup4-4.6.3:
      Successfully uninstalled beautifulsoup4-4.6.3
Installing automake (A dependency for mecab-ko)
Get:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Ign:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Ign:3 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:4 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Get:5 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Hit:6 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:7 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:8 http://ppa.launchpad.net/c2d4u.

In [1]:
removal_list =  "‘, ’, ◇, ‘, ”,  ’, ', ·, \“, ·, △, ●,  , ■, (, ), \", >>, `, /, -,∼,=,ㆍ<,>, .,?, !,【,】, …, ◆,%"

def cleansing_special(sentence):
    # 특수문자를 전처리를 하는 함수
    sentence = re.sub("[.,\'\"’‘”“!?]", "", sentence)
    sentence = re.sub("[^가-힣0-9a-zA-Z\\s]", " ", sentence)
    sentence = re.sub("\s+", " ", sentence)
    
    sentence = sentence.translate(str.maketrans(removal_list, ' '*len(removal_list)))
    sentence = sentence.strip()
    
    return sentence

def cleansing_numbers(sentence):
    # 숫자를 전처리(delexicalization) 하는 함수    
    sentence = re.sub('[0-9]+', 'NUM', sentence)
    sentence = re.sub('NUM\s+', "NUM", sentence)
    sentence = re.sub('[NUM]+', "NUM", sentence)
    
    return sentence

In [8]:
from konlpy.tag import Mecab
mecab = Mecab()

## 데이터셋

In [34]:
path = '/content/drive/MyDrive/data/'

df_books = pd.read_excel(os.path.join(path, 'book_preprocessing.xlsx'))


In [35]:
df_books.tail()

,Unnamed: 0,title,author,pub,price,review,rating,story,img,genre
73503,73503,그녀를 위해,NaN,신영미디어,2880,0,0.0,마르코 고아 조카 안젤리나 사람 필요하다 양육 위 편의 상의 아내 성실하다 착하다 ...,NaN,로맨스
73504,73504,하루만 사랑해 1,NaN,파피루스(디앤씨미디어),8100,1,7.0,날 서원 횡단보도 교통사고 당하다 상대 인천 상고 짱 노랗다 머 민 하루 놈 오히려...,NaN,로맨스
73505,73505,하루만 사랑해 2,NaN,파피루스(디앤씨미디어),8100,0,0.0,날 서원 횡단보도 교통사고 당하다 상대 인천 상고 짱 노랗다 머 민 하루 놈 오히려...,NaN,로맨스
73506,73506,서울 여자와 대구 남자가 만날 때,정민,영상노트,8100,0,0.0,NaN,NaN,로맨스
73507,73507,kiss me...,NaN,현무,7200,2,5.0,NaN,NaN,로맨스


## 데이터 장르별 분리 

In [41]:
df_books.genre.unique()

array(['추리/미스터리', '공포/스릴러', '판타지', '무협', 'SF', '역사', '로맨스'], dtype=object)

In [61]:
df_mystery=df_books[df_books['genre']=='추리/미스터리']

In [47]:
df_horror =df_books[df_books['genre']== '공포/스릴러']
df_fantasy = df_books[df_books['genre']=='판타지']
df_martial = df_books[df_books['genre']=='무협']
df_sf = df_books[df_books['genre']=='SF']
df_history = df_books[df_books['genre']=='역사']
df_romance = df_books[df_books['genre']=='로맨스']

In [51]:
print(df_mystery.shape)
print(df_horror.shape)
print(df_fantasy.shape)
print(df_martial.shape)
print(df_sf.shape)
print(df_history.shape)
print(df_romance.shape)

(3622, 10)
(1521, 10)
(36384, 10)
(13929, 10)
(1040, 10)
(3542, 10)
(13470, 10)


In [63]:
df_mystery = df_mystery.dropna()

In [64]:
df_mystery.shape

(2992, 10)

In [65]:
df_horror =df_horror.dropna()
df_fantasy = df_fantasy.dropna()
df_martial = df_martial.dropna()
df_sf = df_sf.dropna()
df_history = df_history.dropna()
df_romance = df_romance.dropna()


In [66]:
print(df_mystery.shape)
print(df_horror.shape)
print(df_fantasy.shape)
print(df_martial.shape)
print(df_sf.shape)
print(df_history.shape)
print(df_romance.shape)

(2992, 10)
(1407, 10)
(29878, 10)
(11712, 10)
(892, 10)
(2927, 10)
(5358, 10)


In [75]:
clean_words

['단테',
 '클럽',
 '은',
 '간',
 '다빈치',
 '코드',
 '미국',
 '역사',
 '추리',
 '소설',
 '붐',
 '신인',
 '매튜',
 '펄',
 '미국',
 '추리',
 '문학계',
 '일약',
 '스타덤',
 '작품',
 '이',
 '작품',
 '간',
 '즉시',
 '뉴욕',
 '타임스',
 '워싱턴',
 '포스트',
 '뉴스',
 '보스턴',
 '글로브',
 '타임스',
 '보더스',
 '등',
 '베스트셀러']

In [73]:
df_mystery.head()

,Unnamed: 0,title,author,pub,price,review,rating,story,img,genre
0,0,요리코를 위해,노리즈키 린타로,모모,13500,43,9.2,스스로 악마 이자 신 딸 살해 남자 심판 아버지 비극 출발 이자 마지막 끝내 실체 ...,http://image.yes24.com/goods/89392792/L,추리/미스터리
1,1,그녀는 다 계획이 있다,히가시노 게이고,하빌리스,14220,13,9.4,하나 보석 점 살인 사건 과거 미래 사건 리듬 감 터치 게이고 복고 미스터리 교코 ...,http://image.yes24.com/goods/97561319/L,추리/미스터리
2,2,블랙 쇼맨과 이름 없는 마을의 살인,히가시노 게이고,알에이치코리아(RHK),16200,54,9.1,아버지 살해 그것 결혼 소식 직후 순간 비극 시작 대기업 취직 후 약혼자 결혼식 준...,http://image.yes24.com/goods/95712763/L,추리/미스터리
3,3,돌이킬 수 없는 약속,야쿠마루 가쿠,북플라자,13500,217,8.2,제 에드가 란 포상 수상 작가 야쿠 마루 가쿠 미스터리 추리 소설 내 딸 살해 놈 ...,http://image.yes24.com/goods/35552641/L,추리/미스터리
4,4,루팡의 딸,요코제키 다이,북플라자,13500,31,8.5,도둑 집안 대부 변 사체 발견 도둑 집안 대부 이 오 하나 코 손녀 딸 하나 코 자...,http://image.yes24.com/goods/78860364/L,추리/미스터리


In [77]:
from konlpy.tag import  Mecab

In [79]:
mecab = Mecab()
df_mystery_list = list(np.array(df_mystery['story'].tolist()))

In [80]:
print(df_mystery_list)

['스스로 악마 이자 신 딸 살해 남자 심판 아버지 비극 출발 이자 마지막 끝내 실체 파악 수 딸 진실 탐정 모두 누군가 세팅 무대 위 인형 괴물 누구 그 괴물 누구 세 여학생 리코 공원 변 사체 발견', '하나 보석 점 살인 사건 과거 미래 사건 리듬 감 터치 게이고 복고 미스터리 교코 부자 계획 위해 파티 출근 컴패 이 날 하나 보석 점 고객 감사 파티 뒤 직장 동료 에리 호텔 밀실 채 발견 교코 자신', '아버지 살해 그것 결혼 소식 직후 순간 비극 시작 대기업 취직 후 약혼자 결혼식 준비 마 경찰서 온 통 전화 그녀 고향 간다 이제 너 일만 거 던 아버지 전화 생전 마지막 통화 경', '제 에드가 란 포상 수상 작가 야쿠 마루 가쿠 미스터리 추리 소설 내 딸 살해 놈 후 그 약속 내 전 재산 주인공 과거 죄 전 노파 수 약속 미스터리 사건 숨 그', '도둑 집안 대부 변 사체 발견 도둑 집안 대부 이 오 하나 코 손녀 딸 하나 코 자신 신분 채 경찰 집안 장남 카즈마 날 이 괴한 피살 발생', '일본 추리소설 대표 최고 베스트셀러 작가 게이고 데뷔 주년 기념 작 라플라스 마녀 현대문학 번역 간 가도 카 쇼텐 출판사 창립 주년 기념 작 이번 작품 제 란 포상 수상한 데뷔 작 방과 후 이후 통산 단행본 그', '백 퍼센트 사랑 백 퍼센트 헌신 추리 소설 역사상 가장 가장 편의 서사시 게이고 문학 정수 컬 용의자 의 헌신 이 번역 간 일본 문학 전문 번역가 관 씨 자신 번역 대폭 손질 원작 문학 향기 감동 롯', '반전 누구 결말 예측 수 이중 삼중 트릭 스릴 만점 심리전 스포일러 금지 외딴 산장 여덟 명의 남녀 가운데 한밤중 은행 강도 범 침입 인질 인질 차례 탈출 시도 번번이 강도 범과 인질 사이 숨 줄다리기 가운데', '안전가옥 쇼 트 시리즈 두 책 조예은 작가 편집 안전가옥 리지 널 시리즈 첫 책 뉴 서울 파크 젤리 장수 학살 구성 호러 스릴러 작가 연출 편집 더욱 색채 빛 발 폭력 여성 빌런 탄생 그린 초대', '인디언 섬 초대 여덟 명의 손님 웃음

In [82]:
all_sentences = []
for idx, line in enumerate(df_mystery_list):
  print(f"--미스테리소설{idx}번--")

  preprocessed = cleansing_numbers(line)
  preprocessed = cleansing_special(preprocessed)
  #명사
  preprocessed_list = ' '.join(list(set(mecab.nouns(preprocessed))))
  print(preprocessed_list)
  all_sentences.append(preprocessed_list)

스트리밍 출력 내용이 길어서 마지막 5000줄이 삭제되었습니다.
--미스테리소설492번--
이론 실현 스프 탄생 사이언스 기반 미스터리 하나 이야기 기술 미래 장르 과학 등장 소재 배경 픽션
--미스테리소설493번--
셜록 가의 시대 시리즈 초리 인류 소설 홈스 역사 최초 런 개 진진 민간 세상 흥미 추리 작품집 회 판 결정 배스 불문 세계 나라 탐정
--미스테리소설494번--
셜록 가의 시대 시리즈 초리 인류 소설 홈스 역사 최초 런 개 진진 민간 세상 흥미 추리 작품집 회 판 결정 배스 불문 세계 나라 탐정
--미스테리소설495번--
시리즈 로 것 여신 결심 키스 슬립 아스 시작 섬 프란시스 법의학 이야기 도착 충격 세상 미스터리 법 학자 과거 통 재상 편지 차 현대 수 세계 타임 전설 마 보기
--미스테리소설496번--
둘 소설 인상 면 유 것 동안 살인 피해자 우체국 사건 모음 제 조각 집 장 감동 글 수 단편 준비
--미스테리소설497번--
작가 영원 정은수 소설 속 대상 남자 과 문장 사람 추억 일본 추리 미스터리 밤 엘릭시르 전개 공동 이소민 드라마 수상 누군가 간 스토리 원래
--미스테리소설498번--
테 현실 어드벤처 상식 불명 상작 지배 미궁 속 대상 과학 음모 자연 검증 트 인간 조직 사건 제 미스터리 엔터 비밀 온몸 꼭두각시 관련 엘릭시르 수 세계 예측 나라 탐정 존재 전 현상
--미스테리소설499번--
작가 가상 최고 소속 한국 명 상상력 오락 스타일 도시 괴담 월영 추리 협업 무대 학회 협회 콜라보 풍문 독서 시 여름
--미스테리소설500번--
작가 가상 최고 소속 한국 명 상상력 오락 스타일 도시 괴담 월영 추리 협업 무대 학회 협회 콜라보 풍문 독서 시 여름
--미스테리소설501번--
작가 가상 최고 소속 한국 명 상상력 오락 스타일 도시 괴담 월영 추리 협업 무대 학회 협회 콜라보 풍문 독서 시 여름
--미스테리소설502번--
작가 가상 최고 소속 한국 명 상상력 오락 스타일 도시 괴담 월영 추리 협업 무대 학회 협회 콜라보 풍문

## scikit-learn TF-IDF

In [87]:
from sklearn.feature_extraction.text import TfidfVectorizer
from collections import defaultdict  #단어 개수를 세고, 빈도와 리스트 셋 항목 정리리

In [88]:
print(len(all_sentences))

2992


In [89]:
tfidf_vectorizer = TfidfVectorizer()
tfidf_matrix = tfidf_vectorizer.fit_transform(all_sentences)

word2id = defaultdict(lambda : 0)
for idx, feature in enumerate(tfidf_vectorizer.get_feature_names()):
  word2id[feature] = idx

In [93]:
#tfidf_vectorizer.vocabulary_  #어떤 단어가 어떤 번호를 할당받았는지!

In [94]:
tfidf_matrix.shape

(2992, 9512)

In [96]:
for idx, sent in enumerate(all_sentences):
  print(f'---미스테리 소설 {idx} 번---')
  results = [(token, tfidf_matrix[idx, word2id[token]]) for token in sent.split()]
  results.sort(key = lambda element : element[1], reverse = True)
  print(results)
  print("\n")

스트리밍 출력 내용이 길어서 마지막 5000줄이 삭제되었습니다.
---미스테리 소설 1742 번---
[('미혼모', 0.3017061355357008), ('국경', 0.29072690167904514), ('소도시', 0.26936946166460174), ('아무', 0.2642733068213927), ('지목', 0.2487989458203961), ('벨기에', 0.24574415829548008), ('부탁', 0.24574415829548008), ('아가씨', 0.23126113796903858), ('매그레', 0.22930355656829418), ('집안', 0.22396961609532814), ('목격', 0.21785022381840277), ('용의자', 0.19835483456630087), ('여인', 0.18674476865930933), ('실종', 0.18612419449644685), ('발생', 0.18149252169619842), ('아버지', 0.18095117261795826), ('아이', 0.1783539482537338), ('가족', 0.1763963668529894), ('수사', 0.164943034103098), ('남자', 0.15215226500453333), ('사건', 0.08866281266419948), ('사', 0.0), ('것', 0.0), ('베', 0.0), ('집', 0.0), ('접', 0.0)]


---미스테리 소설 1743 번---
[('쪽지', 0.2895770038069887), ('생피', 0.2895770038069887), ('아크', 0.2754493410300377), ('화신', 0.25765059088834524), ('수십', 0.23717027921548345), ('백작', 0.23017828937958068), ('고향', 0.22714654295483108), ('종이', 0.22714654295483108), ('추억', 0.22177545484